In [1]:
import os
import datetime
from pathlib import Path
from collections import defaultdict
import scipy
import random
import numpy as np
import xarray as xr
import pandas as pd
import joblib
import pickle
import xesmf as xe
import glob

import seaborn as sns
import cmocean as cm            # really nice colorbars
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats
import gcsfs 

import requests
import io

In [2]:
# Import data file from google cloud
fs = gcsfs.GCSFileSystem()  # initializes cloud storage interface

input_data_path = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/databases/processed'


In [3]:
fs.ls(input_data_path)

['leap-persistent/ckg-2dxsu/Taylor_data/databases/processed/GLODAPv2_spco2_2023.zarr',
 'leap-persistent/ckg-2dxsu/Taylor_data/databases/processed/HOT_spco2_202312.zarr',
 'leap-persistent/ckg-2dxsu/Taylor_data/databases/processed/LDEOv2019_spco2.zarr',
 'leap-persistent/ckg-2dxsu/Taylor_data/databases/processed/bats_spco2_1988-10-2023-06.zarr']

In [4]:
fs.ls('gs://leap-persistent/ckg-2dxsu/Taylor_data')

['leap-persistent/ckg-2dxsu/Taylor_data/GOBMs',
 'leap-persistent/ckg-2dxsu/Taylor_data/Taylor_inputs',
 'leap-persistent/ckg-2dxsu/Taylor_data/data_products',
 'leap-persistent/ckg-2dxsu/Taylor_data/databases']

In [5]:
BATS_path = input_data_path + '/bats_spco2_1988-10-2023-06.zarr'
HOT_path = input_data_path + '/HOT_spco2_202312.zarr'
LDEO_path = input_data_path + '/LDEOv2019_spco2.zarr'
GLODAP_path = input_data_path + '/GLODAPv2_spco2_2023.zarr'

# SOCCOM_path = 

In [6]:
def BATS_stats(ml_timeseries,recon,start_yr=1991,end_yr=2023,blat=121,blon=115):
    
    # determine appropriate START YEAR # 
    start_yr = max(start_yr, 1991)
    end_yr = min(end_yr, 2023)
    
    # Grab BATS observations:
    bats = xr.open_dataset(BATS_path, engine='zarr')
    
    # Remove duplicates
    bats = bats.groupby('time').mean()
    # Put BATS monthly averages:
    bats_monthly = bats.spco2.to_dataframe().resample('ME').mean()
    # Extract years of interest:
    bats_monthly = bats_monthly.loc[f'{start_yr}-1-06 00:00:00':f'{end_yr}-12-31 23:59:59']
    # 31 50'N 64 10'W ###########     
    
    # Model already monthly #
    pco2_df = ml_timeseries[:,blat,blon].squeeze().to_dataframe()
    if start_yr == 1991:
        pco2_df = pco2_df.loc[f'{start_yr}-10-01 00:00:00':f'{end_yr}-12-31 23:59:59'] 
    else:
        pco2_df = pco2_df.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 23:59:59']    
    
    # get time uniform:
    bats_monthly = bats_monthly.set_index(pco2_df.index)
    
    # correlate where no NANs
    fib = (~np.isnan(bats_monthly.spco2) & ~np.isnan(pco2_df[recon]))
    R = np.corrcoef(pco2_df[recon][fib],bats_monthly.spco2[fib])[0,1]
    STD = np.nanstd(pco2_df[recon][fib])
    BATS_STD = np.nanstd(bats_monthly.spco2[fib])
    
    #RMSE
    RMSE = np.sqrt(np.square(bats_monthly.spco2[fib]-pco2_df[recon][fib]).sum()/(fib.sum()))
    
    return R, STD, RMSE, BATS_STD

In [7]:
def HOT_stats(ml_timeseries,recon,start_yr=1988,end_yr=2022,hlat=112,hlon=22):
    
    # determine appropriate START YEAR # 
    start_yr = max(start_yr, 1988)
    end_yr = min(end_yr, 2022)
    
    # Grab observations
    hot = xr.open_mfdataset(HOT_path, engine= 'zarr') 
    hot = hot.where(hot.spco2>0)
    
    pco2_df = ml_timeseries[:,hlat,hlon].squeeze().to_dataframe()
    if start_yr == 1988:
        pco2_df = pco2_df.loc[f'{start_yr}-10-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    else:
        pco2_df = pco2_df.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
        
    
    # P
    if not isinstance(hot.spco2.to_dataframe().index, pd.DatetimeIndex):
        df_temp = hot.spco2.to_dataframe()
        df_temp.index = pd.to_datetime(df_temp.index)
        hots_monthly = df_temp.resample('ME').mean()  
    else:
        hots_monthly = hot.spco2.to_dataframe().resample('ME').mean()
    hots_monthly = hots_monthly.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    hots_monthly = hots_monthly.set_index(pco2_df.index)
    # Point by point comparison #
    ind = ((~np.isnan(hots_monthly.spco2)) & (hots_monthly.spco2 > 150) & (~np.isnan(pco2_df[recon])))
    R = np.corrcoef(pco2_df[recon][ind],hots_monthly.spco2[ind])[0,1]
    
    STD = np.std(pco2_df[recon][ind])
    HOT_STD = np.nanstd(hots_monthly.spco2[ind])
    
    #RMSE
    RMSE = np.sqrt(np.square(hots_monthly.spco2[ind]-pco2_df[recon][ind]).sum()/ind.sum())
    
    # We want to know the correlation coef, STD of reconstruction, trendline, seasonal cycle:
    return R, STD, RMSE, HOT_STD

In [8]:
def LDEO_stats(ml_gridded,recon,start_yr=1957,end_yr=2019):
    
    # determine appropriate START YEAR # 
    start_yr = max(start_yr, 1957)
    end_yr = min(end_yr, 2019)

    # if recon in ['CSIR_ML6','JENA_MLS','JMA_MLR','MPI_SOMFFN','NIES_FNN','CMEMS_FFNN','spco2','HPD','resid']:
    # lon = ml_gridded.lon
    # lat = ml_gridded.lat
    # else:
        #     lon = ml_gridded.xlon
    #     lat = ml_gridded.ylat
    if 'lon' in ml_gridded.coords:
        lon = ml_gridded.lon
    else:
        lon = ml_gridded.xlon
    
    if 'lat' in ml_gridded.coords:
        lat = ml_gridded.lat
    else:
        lat = ml_gridded.ylat
        
    mtime = ml_gridded.time

    # Load the observations
    ldeo = xr.open_mfdataset(LDEO_path, engine= 'zarr') 
    ldeo_pco2=ldeo.spco2_mean[(start_yr-1985)*12:(end_yr)*12,:,:]
    ldeo_lat=ldeo.lat
    ldeo_lon=ldeo.lon
    ldeo_time = ldeo.time
    
    # Regrid to LDEO's grid #########
    mgrid = xr.Dataset({'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    lgrid = xr.Dataset({'lat':(['lat'],ldeo_lat.values),'lon':(['lon'],ldeo_lon.values)})
    mpco2 = xr.Dataset({recon:(['time','lat','lon'],ml_gridded.values),'time':(['time'],mtime.values),'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    regridder = xe.Regridder(mpco2, lgrid, 'bilinear',periodic = True)
    pco2_new = regridder(ml_gridded)

    ldeo_pco2 = ldeo_pco2.where(((ldeo_pco2<850) & (ldeo_pco2>150)))
    ldeo_pco2 = ldeo_pco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    pco2_new = pco2_new.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    

    # stack
    lpco2 = ldeo_pco2.stack(level=['time','lat','lon'])
    pco2_stack = pco2_new.stack(level=['time','lat','lon'])
    
    indx = ((~np.isnan(lpco2.values)) & (~np.isnan(pco2_stack.values)) & (pco2_stack.values > 50))
    # print("LDEO OBS # =",sum(indx))

    R = np.corrcoef(lpco2[indx],pco2_stack[indx])[0,1]
    LDEO_STD = np.std(lpco2[indx].values)
    STD = np.std(pco2_stack[indx].values)
    #RMSE
    RMSE = np.sqrt(np.square(lpco2[indx].values-pco2_stack[indx].values).sum()/indx.sum())
    
    return R, STD, RMSE, LDEO_STD

In [9]:
def GLODAP_stats(ml_gridded,recon,start_yr=1986,end_yr=2021):
    
    # Reconstructions #
    # if recon in ['CSIR_ML6','JENA_MLS','JMA_MLR','MPI_SOMFFN','NIES_FNN','CMEMS_FFNN','spco2','HPD','resid']:
    # lon = ml_gridded.lon
    # lat = ml_gridded.lat
    # else:
    #     lon = ml_gridded.xlon
    #     lat = ml_gridded.ylat

    if 'lon' in ml_gridded.coords:
        lon = ml_gridded.lon
    else:
        lon = ml_gridded.xlon
    
    if 'lat' in ml_gridded.coords:
        lat = ml_gridded.lat
    else:
        lat = ml_gridded.ylat
        
    start_yr = max(start_yr,1986)
    end_yr = min(end_yr,2021)
    
    mtime = ml_gridded.time
    
    # Load the data #
    glod = xr.open_mfdataset(GLODAP_path, engine= 'zarr') 
    glod_pco2=glod.spco2_mean
    glod_lat=glod.lat
    glod_lon=glod.lon
    glod_time = glod.time
    
    # Deal with NaNs #
    tmp = ml_gridded.where(ml_gridded > 0)
    tmp = tmp.where(tmp < 850)
    
    # Regrid to GLODAP grid #
    mgrid = xr.Dataset({'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    ggrid = xr.Dataset({'lat':(['lat'],glod_lat.values),'lon':(['lon'],glod_lon.values)})
    # mpco2 = xr.Dataset({recon:(['time','lat','lon'、],tmp),'time':(['time'],mtime.values),'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    mpco2 = xr.Dataset({
    recon: (['time', 'lat', 'lon'], tmp.data), 
    'time': (['time'], mtime.values),
    'lat': (['lat'], lat.values),
    'lon': (['lon'], lon.values)
    })
    
    regridder = xe.Regridder(mpco2, ggrid, 'bilinear',periodic = True) #,'periodic'
    pco2_new = regridder(tmp)
    
    # Extract Time #
    glod_pco2 = glod_pco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    glod_pco2 = glod_pco2.where(((glod_pco2 >200) & (glod_pco2 < 600)))
    if start_yr == 1986:
        pco2_new = pco2_new.loc[f'{start_yr}-7-15 00:00:00':f'{end_yr}-12-31 00:00:00']
    else:
        pco2_new = pco2_new.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    
    # Stack
    gpco2_stack = glod_pco2.stack(level=['time','lat','lon'])
    pco2_stack = pco2_new.stack(level=['time','lat','lon'])
    
    # Stats
    indx = ((~np.isnan(gpco2_stack.values)) & (~np.isnan(pco2_stack.values)) & (pco2_stack.values>50))
    R = np.corrcoef(gpco2_stack[indx],pco2_stack[indx])[0,1]
    GLODAP_STD = np.std(gpco2_stack[indx].values)
    STD = np.std(pco2_stack[indx].values)
    #RMSE
    RMSE = np.sqrt(np.square(gpco2_stack[indx].values-pco2_stack[indx].values).sum()/indx.sum())
    
    return R, STD, RMSE, GLODAP_STD

In [10]:
# Set up dataframe #
####################
df = pd.DataFrame() # create empty data frame
df['stats']= ["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]
df.set_index([pd.Index(["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]), 'stats'])


,
,stats
R_BATS,R_BATS
STD_BATS,STD_BATS
RMSE_BATS,RMSE_BATS
R_HOT,R_HOT
STD_HOT,STD_HOT
RMSE_HOT,RMSE_HOT
R_LDEO,R_LDEO
STD_LDEO,STD_LDEO
RMSE_LDEO,RMSE_LDEO


In [11]:
# Set up the time rage we want
start_yr = 1990 
end_yr = 2020

Residual

In [12]:
# update the Residual product. download from zenodo.
# get the url from https://zenodo.org/records/7636890
# resid_url = "https://zenodo.org/records/7636890/files/LDEO-resid_1985-2021.nc?download=1"

### If you want to download from zenodo and not to save it on the GCS, use these code.
# response = requests.get(resid_url)
# response.raise_for_status()  # make sure download successfully
# # Save to temporary memory
# data_in_memory = io.BytesIO(response.content)
# # open as xarray
# ds = xr.open_dataset(data_in_memory, engine='h5netcdf', chunks={})
# gridded = ds.pco2
# # resid_product = ds.pco2
# recon = 'pco2' #just what the variable is called.
####

# Here we use the 2022 version from LEAP Pangeo
resid_url = 'gs://leap-persistent/galenmckinley/reconstructions/pCO2_LEAP_XGBoost-fco2-residual-reconstructed_198201-202212.zarr'
# open as xarray
ds = xr.open_mfdataset(resid_url,engine= 'zarr')

gridded = ds.fco2_reconstructed
recon = 'fco2_reconstructed' #just what the variable is called.

In [13]:
R_BATS, STD_BATS, RMSE_BATS, b_obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, h_obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, s_obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, l_obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, g_obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xesmf/frontend.py:718: UserWarning: Using dimensions ('ylat', 'xlon') from data variable fco2_reconstructed as the horizontal dimensions for the regridding.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.12/site-packages/xesmf/frontend.py:718: UserWarning: Using dimensions ('ylat', 'xlon') from data variable fco2_reconstructed as the horizontal dimensions for the regridding.
  warnings.warn(


In [14]:
df['Residual']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
#put the observered on there too.
df['observed']=[1,b_obs,0,1,h_obs,0,1,l_obs,0,1,g_obs,0] #not sure exactly but doing what val does with this

In [15]:
df.head()

,stats,Residual,observed
0,R_BATS,0.933359,1.000000
1,STD_BATS,32.600659,36.542407
2,RMSE_BATS,13.203849,0.000000
3,R_HOT,0.927477,1.000000
4,STD_HOT,21.448400,20.788085


HPD product 

In [16]:
# update and grab HPD product. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
HPD_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_LDEO-HPD_1959-2022.nc?download=1"
directory = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_LDEO-HPD_1959-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(HPD_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products/GCB-2023_dataprod_LDEO-HPD_1959-2022.zarr. Loading from GCS.


In [17]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2' #just what the variable is called.

In [18]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


In [19]:
df['HPD']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

,stats,Residual,observed,HPD
0,R_BATS,0.933359,1.000000,0.941681
1,STD_BATS,32.600659,36.542407,32.566799
2,RMSE_BATS,13.203849,0.000000,12.503740
3,R_HOT,0.927477,1.000000,0.938260
4,STD_HOT,21.448400,20.788085,21.322348
5,RMSE_HOT,8.783743,0.000000,7.696231
6,R_LDEO,0.961666,1.000000,0.927416
7,STD_LDEO,34.849722,37.604685,34.671730
8,RMSE_LDEO,10.444084,0.000000,14.115043
9,R_GLODAP,0.900089,1.000000,0.896611


CMEMS-LSCE-FFNN

In [20]:
# CMEMS. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
CMEMS_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_CMEMS-LSCE-FFNN_1990-2022.nc?download=1"
directory = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_CMEMS-LSCE-FFNN_1990-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(CMEMS_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products/GCB-2023_dataprod_CMEMS-LSCE-FFNN_1990-2022.zarr. Loading from GCS.


In [21]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

#CMEMS is too short (starts in 1990) so need to add blank months before 1990 to make it the same size?
gridded2 = gridded.loc['1992-1-01 00:00:00':'1999-12-31 00:00:00']
gridded2 = gridded2.where(gridded2 < 0) #this turns anything that is 0 or greater (not true in this equation) to nan.

earlytime = pd.date_range(start='1982-01-01', end='1989-12-31',freq='MS') + np.timedelta64(14, 'D') 

gridded2 = gridded2.assign_coords(time=earlytime) #overwrite time dimension to be midmonth

gridded2.coords['lon'] = (gridded2.coords['lon'] + 180) % 360 - 180
gridded2 = gridded2.sortby(gridded2.lon)

#concat gridded2 and gridded for CMEMS:
gridded_long = xr.concat([gridded2,gridded],dim='time')

recon = 'sfco2' #need this to work with *****_stats functions above. But not sure what to make it or how to not need it.

In [22]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


In [23]:
df['CMEMS']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

,stats,Residual,observed,HPD,CMEMS
0,R_BATS,0.933359,1.000000,0.941681,0.944955
1,STD_BATS,32.600659,36.542407,32.566799,31.540739
2,RMSE_BATS,13.203849,0.000000,12.503740,12.956892
3,R_HOT,0.927477,1.000000,0.938260,0.907752
4,STD_HOT,21.448400,20.788085,21.322348,21.138533
5,RMSE_HOT,8.783743,0.000000,7.696231,9.378827
6,R_LDEO,0.961666,1.000000,0.927416,0.871714
7,STD_LDEO,34.849722,37.604685,34.671730,33.044003
8,RMSE_LDEO,10.444084,0.000000,14.115043,18.362817
9,R_GLODAP,0.900089,1.000000,0.896611,0.851241


JENA-MLS

In [24]:
# JENA. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
JENA_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_JENA-MLS_1957-2022.nc?download=1"
directory = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_JENA-MLS_1957-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(JENA_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products/GCB-2023_dataprod_JENA-MLS_1957-2022.zarr. Loading from GCS.


In [25]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)
recon = 'sfco2'  #just what the variable is called.

In [26]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


In [27]:
df['JENA']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

,stats,Residual,observed,HPD,CMEMS,JENA
0,R_BATS,0.933359,1.000000,0.941681,0.944955,0.903496
1,STD_BATS,32.600659,36.542407,32.566799,31.540739,35.462399
2,RMSE_BATS,13.203849,0.000000,12.503740,12.956892,16.007577
3,R_HOT,0.927477,1.000000,0.938260,0.907752,0.887936
4,STD_HOT,21.448400,20.788085,21.322348,21.138533,23.079691
5,RMSE_HOT,8.783743,0.000000,7.696231,9.378827,11.360949
6,R_LDEO,0.961666,1.000000,0.927416,0.871714,0.899077
7,STD_LDEO,34.849722,37.604685,34.671730,33.044003,35.118721
8,RMSE_LDEO,10.444084,0.000000,14.115043,18.362817,17.398994
9,R_GLODAP,0.900089,1.000000,0.896611,0.851241,0.820785


JMA-MLR

In [28]:
# JMA. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
JMA_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_JMA-MLR_1985-2022.nc?download=1"
directory = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_JMA-MLR_1985-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(JMA_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products/GCB-2023_dataprod_JMA-MLR_1985-2022.zarr. Loading from GCS.


In [29]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2'  #just what the variable is called.

In [30]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


In [31]:
df['JMA']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

,stats,Residual,observed,HPD,CMEMS,JENA,JMA
0,R_BATS,0.933359,1.000000,0.941681,0.944955,0.903496,0.935217
1,STD_BATS,32.600659,36.542407,32.566799,31.540739,35.462399,28.029915
2,RMSE_BATS,13.203849,0.000000,12.503740,12.956892,16.007577,15.324630
3,R_HOT,0.927477,1.000000,0.938260,0.907752,0.887936,0.901864
4,STD_HOT,21.448400,20.788085,21.322348,21.138533,23.079691,25.622540
5,RMSE_HOT,8.783743,0.000000,7.696231,9.378827,11.360949,11.321679
6,R_LDEO,0.961666,1.000000,0.927416,0.871714,0.899077,0.794457
7,STD_LDEO,34.849722,37.604685,34.671730,33.044003,35.118721,33.269375
8,RMSE_LDEO,10.444084,0.000000,14.115043,18.362817,17.398994,24.246742
9,R_GLODAP,0.900089,1.000000,0.896611,0.851241,0.820785,0.744834


NIES-ML3

In [32]:
# NIES. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
NIES_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_NIES-ML3_1980-2022.nc?download=1"
directory = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_NIES-ML3_1980-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(NIES_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products/GCB-2023_dataprod_NIES-ML3_1980-2022.zarr. Loading from GCS.


In [33]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00'] 

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2'  #just what the variable is called.

In [34]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


In [35]:
df['NIES']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

,stats,Residual,observed,HPD,CMEMS,JENA,JMA,NIES
0,R_BATS,0.933359,1.000000,0.941681,0.944955,0.903496,0.935217,0.946166
1,STD_BATS,32.600659,36.542407,32.566799,31.540739,35.462399,28.029915,31.667589
2,RMSE_BATS,13.203849,0.000000,12.503740,12.956892,16.007577,15.324630,12.416096
3,R_HOT,0.927477,1.000000,0.938260,0.907752,0.887936,0.901864,0.927502
4,STD_HOT,21.448400,20.788085,21.322348,21.138533,23.079691,25.622540,19.448282
5,RMSE_HOT,8.783743,0.000000,7.696231,9.378827,11.360949,11.321679,7.950455
6,R_LDEO,0.961666,1.000000,0.927416,0.871714,0.899077,0.794457,0.871910
7,STD_LDEO,34.849722,37.604685,34.671730,33.044003,35.118721,33.269375,31.821764
8,RMSE_LDEO,10.444084,0.000000,14.115043,18.362817,17.398994,24.246742,18.779421
9,R_GLODAP,0.900089,1.000000,0.896611,0.851241,0.820785,0.744834,0.848839


SOM-FFN

In [36]:
# SOMFFN. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
SOMFFN_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_SOM-FFN_1982-2022.nc?download=1"
directory = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_SOM-FFN_1982-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(SOMFFN_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products/GCB-2023_dataprod_SOM-FFN_1982-2022.zarr. Loading from GCS.


In [37]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00'] 
#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2'  #just what the variable is called.

In [38]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


In [39]:
df['SOMFFN']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

,stats,Residual,observed,HPD,CMEMS,JENA,JMA,NIES,SOMFFN
0,R_BATS,0.933359,1.000000,0.941681,0.944955,0.903496,0.935217,0.946166,0.927867
1,STD_BATS,32.600659,36.542407,32.566799,31.540739,35.462399,28.029915,31.667589,29.328060
2,RMSE_BATS,13.203849,0.000000,12.503740,12.956892,16.007577,15.324630,12.416096,14.517769
3,R_HOT,0.927477,1.000000,0.938260,0.907752,0.887936,0.901864,0.927502,0.896313
4,STD_HOT,21.448400,20.788085,21.322348,21.138533,23.079691,25.622540,19.448282,16.921322
5,RMSE_HOT,8.783743,0.000000,7.696231,9.378827,11.360949,11.321679,7.950455,9.461876
6,R_LDEO,0.961666,1.000000,0.927416,0.871714,0.899077,0.794457,0.871910,0.856991
7,STD_LDEO,34.849722,37.604685,34.671730,33.044003,35.118721,33.269375,31.821764,31.250069
8,RMSE_LDEO,10.444084,0.000000,14.115043,18.362817,17.398994,24.246742,18.779421,20.887692
9,R_GLODAP,0.900089,1.000000,0.896611,0.851241,0.820785,0.744834,0.848839,0.812839


UoEX_WATv2

In [40]:
# UoEX_WAT. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
WAT_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_UoEX_WATv2_1985-2022.nc?download=1"
directory = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_UoEX_WATv2_1985-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(WAT_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products/GCB-2023_dataprod_UoEX_WATv2_1985-2022.zarr. Loading from GCS.


In [41]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2'  #just what the variable is called.

In [42]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


In [43]:
df['UoEX_WAT']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

,stats,Residual,observed,HPD,CMEMS,JENA,JMA,NIES,SOMFFN,UoEX_WAT
0,R_BATS,0.933359,1.000000,0.941681,0.944955,0.903496,0.935217,0.946166,0.927867,0.926059
1,STD_BATS,32.600659,36.542407,32.566799,31.540739,35.462399,28.029915,31.667589,29.328060,29.308624
2,RMSE_BATS,13.203849,0.000000,12.503740,12.956892,16.007577,15.324630,12.416096,14.517769,15.152214
3,R_HOT,0.927477,1.000000,0.938260,0.907752,0.887936,0.901864,0.927502,0.896313,0.893564
4,STD_HOT,21.448400,20.788085,21.322348,21.138533,23.079691,25.622540,19.448282,16.921322,17.754431
5,RMSE_HOT,8.783743,0.000000,7.696231,9.378827,11.360949,11.321679,7.950455,9.461876,10.450091
6,R_LDEO,0.961666,1.000000,0.927416,0.871714,0.899077,0.794457,0.871910,0.856991,0.828947
7,STD_LDEO,34.849722,37.604685,34.671730,33.044003,35.118721,33.269375,31.821764,31.250069,32.297768
8,RMSE_LDEO,10.444084,0.000000,14.115043,18.362817,17.398994,24.246742,18.779421,20.887692,22.090448
9,R_GLODAP,0.900089,1.000000,0.896611,0.851241,0.820785,0.744834,0.848839,0.812839,0.761902


OceanSODA-ETHZ

In [44]:
# OceanSODA-ETHZ. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
OceanSODA_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_OceanSODA-ETHZ_1982-2022.nc?download=1"
directory = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_OceanSODA-ETHZ_1982-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(OceanSODA_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/data_products/GCB-2023_dataprod_OceanSODA-ETHZ_1982-2022.zarr. Loading from GCS.


In [45]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2'  #just what the variable is called.

In [46]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


In [47]:
df['OceanSODA']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

,stats,Residual,observed,HPD,CMEMS,JENA,JMA,NIES,SOMFFN,UoEX_WAT,OceanSODA
0,R_BATS,0.933359,1.000000,0.941681,0.944955,0.903496,0.935217,0.946166,0.927867,0.926059,0.942021
1,STD_BATS,32.600659,36.542407,32.566799,31.540739,35.462399,28.029915,31.667589,29.328060,29.308624,30.927252
2,RMSE_BATS,13.203849,0.000000,12.503740,12.956892,16.007577,15.324630,12.416096,14.517769,15.152214,12.803260
3,R_HOT,0.927477,1.000000,0.938260,0.907752,0.887936,0.901864,0.927502,0.896313,0.893564,0.910146
4,STD_HOT,21.448400,20.788085,21.322348,21.138533,23.079691,25.622540,19.448282,16.921322,17.754431,20.025301
5,RMSE_HOT,8.783743,0.000000,7.696231,9.378827,11.360949,11.321679,7.950455,9.461876,10.450091,8.683174
6,R_LDEO,0.961666,1.000000,0.927416,0.871714,0.899077,0.794457,0.871910,0.856991,0.828947,0.873803
7,STD_LDEO,34.849722,37.604685,34.671730,33.044003,35.118721,33.269375,31.821764,31.250069,32.297768,33.519089
8,RMSE_LDEO,10.444084,0.000000,14.115043,18.362817,17.398994,24.246742,18.779421,20.887692,22.090448,19.357073
9,R_GLODAP,0.900089,1.000000,0.896611,0.851241,0.820785,0.744834,0.848839,0.812839,0.761902,0.849145


Models

In [48]:
# list of all models and their download url
# edit if necessary
models = ['ACCESS', 'CESM_ETHZ', 'CNRM_ESM2', 'FESOM2_REcoM', 'IPSL', 'MOM6_Princeton', 'MPIOM_HAMOCC', 'MRI_ESM2_2', 
          'NEMO_PlankTOM12', 'NorESM_OC1_2']
urls = ['https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_ACCESS_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_CESM_ETHZ_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_CNRM_ESM2_1_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_FESOM2_REcoM_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_IPSL_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_MOM6_Princeton_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_MPIOM_HAMOCC_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_MRI_ESM2_2_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_NEMO_PlankTOM12_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_NorESM_OC1_2_1959-2022.nc?download=1'
       ]

In [49]:
directory = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/GOBMs'
for model, url in zip(models, urls):
    file_name = url.split('/')[-1].split('.nc')[0]
    save_path = f'{directory}/{file_name}.zarr'
    
    fs = gcsfs.GCSFileSystem()
    
    # Check if we already have the file on GCS
    if fs.exists(save_path):
        print(f"File already exists at {save_path}. Loading from GCS.")
        ds = xr.open_mfdataset(save_path, engine = 'zarr')
    else:
        print(f"File does not exist at {save_path}. Downloading and saving.")
        response = requests.get(url)
        if response.status_code == 200:
            with open('temp.nc', 'wb') as temp_file:
                temp_file.write(response.content)
            ds = xr.open_dataset('temp.nc')
            ds.to_zarr(save_path, mode='w')
            print(f"Successfully downloaded and saved to {save_path}")
            os.remove('temp.nc')
        else:
            print(f"Error! Could not download the file: {response.status_code}")
    
    # gridded = ds.sfco2
    # gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    gridded = ds.sfco2.sel(simulation='A').sel(time=slice(f'{start_yr}-01-01T00:00:00', f'{end_yr}-12-31T23:59:59'))
    
    #regrid longitude
    gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
    gridded = gridded.sortby(gridded.lon)
    
    recon = 'sfco2'  #just what the variable is called.

    R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)
    R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)
    R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)
    R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

    df[model]=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]

df

File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/GOBMs/GCB-2023_OceanModel_ACCESS_1959-2022.zarr. Loading from GCS.


/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/GOBMs/GCB-2023_OceanModel_CESM_ETHZ_1959-2022.zarr. Loading from GCS.


/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/numpy/core/_methods.py:152: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/GOBMs/GCB-2023_OceanModel_CNRM_ESM2_1_1959-2022.zarr. Loading from GCS.


...

File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/GOBMs/GCB-2023_OceanModel_NEMO_PlankTOM12_1959-2022.zarr. Loading from GCS.


/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


File already exists at gs://leap-persistent/ckg-2dxsu/Taylor_data/GOBMs/GCB-2023_OceanModel_NorESM_OC1_2_1959-2022.zarr. Loading from GCS.


/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)
/srv/conda/envs/notebook/lib/python3.12/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 16.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (45, 90).
  result_var = func(*data_vars)


,stats,Residual,observed,HPD,CMEMS,JENA,JMA,NIES,SOMFFN,UoEX_WAT,...,ACCESS,CESM_ETHZ,CNRM_ESM2,FESOM2_REcoM,IPSL,MOM6_Princeton,MPIOM_HAMOCC,MRI_ESM2_2,NEMO_PlankTOM12,NorESM_OC1_2
0,R_BATS,0.933359,1.000000,0.941681,0.944955,0.903496,0.935217,0.946166,0.927867,0.926059,...,0.942465,9.381828e-01,0.908234,0.943897,0.939577,0.940143,0.923830,0.938246,0.935089,0.872860
1,STD_BATS,32.600659,36.542407,32.566799,31.540739,35.462399,28.029915,31.667589,29.328060,29.308624,...,38.962708,3.836979e+01,38.018192,44.162376,37.463646,43.754738,49.398125,43.214527,40.584278,29.514166
2,RMSE_BATS,13.203849,0.000000,12.503740,12.956892,16.007577,15.324630,12.416096,14.517769,15.152214,...,18.394082,1.329303e+01,20.273365,16.657253,17.455484,15.773159,28.392774,17.330075,19.148208,18.273994
3,R_HOT,0.927477,1.000000,0.938260,0.907752,0.887936,0.901864,0.927502,0.896313,0.893564,...,0.910504,9.363017e-01,0.900993,0.936544,0.931889,0.935528,0.911250,0.932047,0.918513,0.931659
4,STD_HOT,21.448400,20.788085,21.322348,21.138533,23.079691,25.622540,19.448282,16.921322,17.754431,...,22.718719,2.207301e+01,21.716291,22.286270,22.463205,22.691120,27.425406,22.552292,22.419975,23.503265
5,RMSE_HOT,8.783743,0.000000,7.696231,9.378827,11.360949,11.321679,7.950455,9.461876,10.450091,...,16.180429,8.171962e+00,15.157043,9.103994,8.471633,13.730531,22.504390,11.078929,13.545771,28.943042
6,R_LDEO,0.961666,1.000000,0.927416,0.871714,0.899077,0.794457,0.871910,0.856991,0.828947,...,0.507896,3.072412e-03,0.563564,0.597834,0.607312,0.680216,0.562971,0.538377,0.560197,0.577771
7,STD_LDEO,34.849722,37.604685,34.671730,33.044003,35.118721,33.269375,31.821764,31.250069,32.297768,...,35.035988,inf,36.672890,44.563839,38.125362,39.524727,42.431969,36.835346,39.221794,34.292435
8,RMSE_LDEO,10.444084,0.000000,14.115043,18.362817,17.398994,24.246742,18.779421,20.887692,22.090448,...,39.557314,1.351884e+36,36.377408,37.894598,35.039783,31.498500,38.484923,37.134641,36.374214,35.339268
9,R_GLODAP,0.900089,1.000000,0.896611,0.851241,0.820785,0.744834,0.848839,0.812839,0.761902,...,0.502281,6.534825e-01,0.563863,0.631356,0.579682,0.655112,0.630960,0.504242,0.546568,0.605479


In [50]:
# to show the statistics of Models
df.iloc[:,11:]

,ACCESS,CESM_ETHZ,CNRM_ESM2,FESOM2_REcoM,IPSL,MOM6_Princeton,MPIOM_HAMOCC,MRI_ESM2_2,NEMO_PlankTOM12,NorESM_OC1_2
0,0.942465,9.381828e-01,0.908234,0.943897,0.939577,0.940143,0.923830,0.938246,0.935089,0.872860
1,38.962708,3.836979e+01,38.018192,44.162376,37.463646,43.754738,49.398125,43.214527,40.584278,29.514166
2,18.394082,1.329303e+01,20.273365,16.657253,17.455484,15.773159,28.392774,17.330075,19.148208,18.273994
3,0.910504,9.363017e-01,0.900993,0.936544,0.931889,0.935528,0.911250,0.932047,0.918513,0.931659
4,22.718719,2.207301e+01,21.716291,22.286270,22.463205,22.691120,27.425406,22.552292,22.419975,23.503265
5,16.180429,8.171962e+00,15.157043,9.103994,8.471633,13.730531,22.504390,11.078929,13.545771,28.943042
6,0.507896,3.072412e-03,0.563564,0.597834,0.607312,0.680216,0.562971,0.538377,0.560197,0.577771
7,35.035988,inf,36.672890,44.563839,38.125362,39.524727,42.431969,36.835346,39.221794,34.292435
8,39.557314,1.351884e+36,36.377408,37.894598,35.039783,31.498500,38.484923,37.134641,36.374214,35.339268
9,0.502281,6.534825e-01,0.563863,0.631356,0.579682,0.655112,0.630960,0.504242,0.546568,0.605479


Save to GCS

In [51]:
# Write out Files #

# Get current time
now = datetime.datetime.now()
current_year = now.year
current_month = str(now.month).zfill(2)
current_day = str(now.day).zfill(2)

output_path = 'gs://leap-persistent/ckg-2dxsu/Taylor_data/Taylor_inputs'

# Edit the file name here
# if we set the time range from earlier than 1986 to later than 2022, then we assume it outputs the full time rage version
if (start_yr <= 1986 & end_yr >= 2022) or (start_yr == None and end_yr == None): 
    csv_out_path = output_path + f'/allproducts_pCO2_Taylor_stats_Full_version{current_year}{current_month}{current_day}.csv'
else:
    csv_out_path = output_path + f'/allproducts_pCO2_Taylor_stats_version{current_year}{current_month}{current_day}_{start_yr}-{end_yr}.csv'


with fs.open(csv_out_path, 'w') as f:
    df.to_csv(f, index=False)

# Check if saved sucessfully
if fs.exists(csv_out_path):
    print(f"Successfully saved to {csv_out_path}")
else:
    print(f"Failed to save to {csv_out_path}")

Successfully saved to gs://leap-persistent/ckg-2dxsu/Taylor_data/Taylor_inputs/allproducts_pCO2_Taylor_stats_version20240825_1990-2020.csv


In [52]:
fs.ls(output_path)

['leap-persistent/ckg-2dxsu/Taylor_data/Taylor_inputs/allproducts_pCO2_Taylor_stats_version20240816_1990-2020.csv',
 'leap-persistent/ckg-2dxsu/Taylor_data/Taylor_inputs/allproducts_pCO2_Taylor_stats_version20240823_1990-2020.csv',
 'leap-persistent/ckg-2dxsu/Taylor_data/Taylor_inputs/allproducts_pCO2_Taylor_stats_version20240825_1990-2020.csv']